In [3]:
from typing import TypedDict, Literal, Annotated
from operator import add
import json
from dotenv import load_dotenv

from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# ----------------------------
# LLM Configuration
# ----------------------------
llm = ChatOpenAI(model="gpt-4o")



# ----------------------------
# Prompt Templates
# ----------------------------

generate_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a viral Twitter content creator.
Create a highly engaging, witty, funny, and potentially viral tweet.
Keep it under 280 characters.
Make it sharp, clever, and shareable.
If feedback is provided, improve the tweet based on that feedback."""
        ),
        (
            "human",
            """Topic: {topic}
Previous feedback (if any): {feedback}

Generate the improved tweet:"""
        ),
    ]
)

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a viral Twitter growth expert and humor analyst.

Evaluate the tweet based on:
1. Humor
2. Virality potential
3. Engagement likelihood
4. Clarity and punch

Return your response strictly in JSON format:

{{
  "score": <integer 1-10>,
  "feedback": "<clear actionable improvement suggestions>"
}}

Be critical. Only give 8 or above if it's genuinely strong."""
        ),
        (
            "human",
            """Tweet:
{tweet}"""
        ),
    ]
)

generate_chain = generate_prompt | llm | StrOutputParser()
reflection_chain = reflection_prompt | llm | StrOutputParser()

print("----------GENERATION CHAIN ---------")
print(generate_chain)
print();print()

# ----------------------------
# State Definition
# ----------------------------
class TweetState(TypedDict):
    topic: str
    tweet: str
    feedback: str
    score: int
    count: Annotated[int, add]  # reducer auto-adds
    
# ----------------------------
# Nodes
# ----------------------------

def generate_tweet(state: TweetState):
    tweet = generate_chain.invoke(
        {
            "topic": state["topic"],
            "feedback": state.get("feedback", "")
        }
    )

    print(f"\n=== GENERATION ROUND {state.get('count', 0) + 1} ===")
    print(tweet)

    return {
        "tweet": tweet,
        "count": 1  # reducer will increment
    }

response = generate_tweet({
     "topic":"Lagos Hustle",
     "count":1
}) 

print();print()
print("=================TEST GENERATION=====================")
print(response)

----------GENERATION CHAIN ---------
first=ChatPromptTemplate(input_variables=['feedback', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a viral Twitter content creator.\nCreate a highly engaging, witty, funny, and potentially viral tweet.\nKeep it under 280 characters.\nMake it sharp, clever, and shareable.\nIf feedback is provided, improve the tweet based on that feedback.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['feedback', 'topic'], input_types={}, partial_variables={}, template='Topic: {topic}\nPrevious feedback (if any): {feedback}\n\nGenerate the improved tweet:'), additional_kwargs={})]) middle=[] last=ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'text_inputs': True, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image